In [ ]:
#A client of mine, Margot, has always been a fan of the accent down under. She has always wanted to move there and finally found the courage to. 
#Margot has a friend in Hawthorn, Melbourne. While she wants to have a friend nearby in the foreign country, she also wants to live in a neighborhood that fits her lifestyle.
#She asked me to help her explore neighborhoods in Hawthorn so that she can make a well informed decision.
#Margot's criteria for her new ideal neighborhood are: a variety of yoga studio, gyms, cafes and bars. 

In [ ]:
#I will be using location data of Victoria (State in Melbourne, where Hawthorn is) in the below link to identify different neighborhoods in Hawthorn.
# https://www.geonames.org/postalcode-search.html?q=&country=AU&adminCode1=VIC

In [ ]:
#I will
#1. pass this info to Foursquare and identify popular venues in each area.
#2. summarize top 10 most popular venues in each neighbourhood.
#3. cluster neighborhoods 
#4. advice which neighborhoods would fit Margot's lifestyle best.

In [17]:
!pip install bs4
!conda install -c conda-forge folium=0.5.0

Solving environment: done

# All requested packages already installed.



In [56]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import numpy as np

In [19]:
url="https://www.geonames.org/postalcode-search.html?q=&country=AU&adminCode1=VIC"
html_data=requests.get(url).text

In [20]:
soup=BeautifulSoup(html_data,"html5lib")

In [21]:
mel_table=soup.find_all("table",class_='restable')
mel_table

[<table class="restable">
 <tbody><tr><th></th><th>Place</th><th>Code</th><th>Country</th><th>Admin1</th><th>Admin2</th><th>Admin3</th></tr>
 <tr><td><small>1</small></td><td>Melbourne</td><td>3000</td><td>Australia</td><td>Victoria</td><td>MELBOURNE CITY</td><td></td></tr><tr><td></td><td colspan="6">   <a href="/maps/browse_-37.813_144.961.html" rel="nofollow"><small>-37.813/144.961</small></a></td></tr>
 <tr class="odd"><td><small>2</small></td><td>East Melbourne</td><td>3002</td><td>Australia</td><td>Victoria</td><td>MELBOURNE CITY</td><td></td></tr><tr class="odd"><td></td><td colspan="6">   <a href="/maps/browse_-37.813_144.984.html" rel="nofollow"><small>-37.813/144.984</small></a></td></tr>
 <tr><td><small>3</small></td><td>West Melbourne</td><td>3003</td><td>Australia</td><td>Victoria</td><td>MELBOURNE CITY</td><td></td></tr><tr><td></td><td colspan="6">   <a href="/maps/browse_-37.809_144.947.html" rel="nofollow"><small>-37.809/144.947</small></a></td></tr>
 <tr class="odd"><

In [22]:
table_rows=mel_table[0].find_all("tr")
l=[]
for tr in table_rows:
    col=tr.find_all("td")
    row=[tr.text for tr in col]
    l.append(row)
    
all_data=pd.DataFrame(l,columns=["index","Suburb","Code","Country","State","City","None"])
all_data

,index,Suburb,Code,Country,State,City,None
0,None,None,None,None,None,None,None
1,1,Melbourne,3000,Australia,Victoria,MELBOURNE CITY,
2,,-37.813/144.961,None,None,None,None,None
3,2,East Melbourne,3002,Australia,Victoria,MELBOURNE CITY,
4,,-37.813/144.984,None,None,None,None,None
...,...,...,...,...,...,...,...
397,199,Hillside,3037,Australia,Victoria,FOOTSCRAY,
398,,-37.691/144.743,None,None,None,None,None
399,200,Taylors Hill,3037,Australia,Victoria,,
400,,-37.715/144.753,None,None,None,None,None


In [23]:
is_place=(all_data["index"]!="")&(all_data["index"]!=None)
place_data=all_data[is_place]
place_data2=place_data.dropna()
place_data2.reset_index(inplace=True)
place_data2

,level_0,index,Suburb,Code,Country,State,City,None
0,1,1,Melbourne,3000,Australia,Victoria,MELBOURNE CITY,
1,3,2,East Melbourne,3002,Australia,Victoria,MELBOURNE CITY,
2,5,3,West Melbourne,3003,Australia,Victoria,MELBOURNE CITY,
3,7,4,Melbourne,3004,Australia,Victoria,MELBOURNE CITY,
4,9,5,Docklands,3008,Australia,Victoria,MELBOURNE CITY,
...,...,...,...,...,...,...,...,...
195,391,196,Travancore,3032,Australia,Victoria,FOOTSCRAY,
196,393,197,Avondale Heights,3034,Australia,Victoria,FOOTSCRAY,
197,395,198,Sydenham,3037,Australia,Victoria,FOOTSCRAY,
198,397,199,Hillside,3037,Australia,Victoria,FOOTSCRAY,


In [38]:
place_data3 = place_data2['Suburb']
neighborhood = place_data3.tolist()

In [30]:
#identify latitude and longitude for each neighbourhood
geolocator = Nominatim(user_agent="mygeocoder")
coordinates_of_neighborhood = []
for i in neighborhood:
    location = geolocator.geocode(i.format(i))
    if location:
        coordinates_of_neighborhood.append([i, location.latitude, location.longitude])

In [31]:
#pass latitude and longitude into dataframe
data_frame_of_coordinates = pd.DataFrame(coordinates_of_neighborhood, columns=["Suburb", "Latitude","Longitude"])
data_frame_of_coordinates

,Suburb,Latitude,Longitude
0,Melbourne,-37.814218,144.963161
1,East Melbourne,-37.815588,144.982555
2,West Melbourne,28.042227,-80.664094
3,Melbourne,-37.814218,144.963161
4,Docklands,-37.817870,144.938654
...,...,...,...
195,Travancore,-37.780755,144.935503
196,Avondale Heights,-37.761454,144.862141
197,Sydenham,51.426766,-0.052388
198,Hillside,41.877808,-87.902840


In [27]:
address = 'Melbourne City, AUS'

geolocator = Nominatim(user_agent="aus_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Melbourne City are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Melbourne City are -37.8166793, 144.9924662.


In [32]:
#pull map for Melbourne City, Australia
map_aus = folium.Map(location=[latitude, longitude], zoom_start=12)
label = "Melbourne City, AUS"
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude, longitude],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_aus)
    
map_aus

In [46]:
hawthorn_list=['collingwood','Abbotsford','Ivanhoe','Kew','Balwyn','Hawthorn','Burwood','Richmond','Toorak','Docklands','Armadale','Malvern','Elwood']
hawthorn_data = data_frame_of_coordinates[data_frame_of_coordinates['Suburb'].isin(hawthorn_list)]
hawthorn_data.head()

,Suburb,Latitude,Longitude
4,Docklands,-37.817870,144.938654
45,Abbotsford,49.052116,-122.329479
52,Ivanhoe,44.463229,-96.247372
55,Kew,51.478126,-0.296934
56,Balwyn,-37.809174,145.083368


In [33]:
# The code was removed by Watson Studio for sharing.

In [34]:
venues = []
query = ['yoga']
for latitude, longtitude, name in zip(data_frame_of_coordinates['Latitude'], data_frame_of_coordinates['Longitude'], data_frame_of_coordinates['Suburb']):
    url = f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={latitude},{longtitude}&query={query}&radius={radius}&limit={LIMIT}"
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append((latitude, 
                       longtitude,
                       venue['venue']['name'],
                       venue['venue']['location']['lat'],
                       venue['venue']['location']['lng'],
                       venue['venue']['categories'][0]['name']))

In [32]:
#identify list gyms in Melbourne
gym_data_frame = pd.DataFrame(venues)
gym_data_frame.columns = ['Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
gym_data_frame

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,-37.814218,144.963161,Breathe Yoga & Pilates,-37.814905,144.964936,Yoga Studio
1,-37.814218,144.963161,Barre Body,-37.816009,144.968455,Yoga Studio
2,-37.814218,144.963161,RACV Club - Fitness Centre,-37.815656,144.959660,Yoga Studio
3,-37.814218,144.963161,Virgin Active Health Club,-37.818806,144.955917,Gym / Fitness Center
4,-37.814218,144.963161,Virgin Active Health Club,-37.812063,144.968810,Gym / Fitness Center
...,...,...,...,...,...,...
212,-34.944479,138.647296,YUJ Life Yoga Community,-34.960550,138.642340,Yoga Studio
213,-37.798798,144.776051,Carja Yoga,-37.783757,144.773701,Yoga Studio
214,-37.780755,144.935503,Centre for Mind & Movement,-37.766450,144.924230,Gym / Fitness Center
215,-33.914743,151.166107,Lifestyle Fitness Australia,-33.906573,151.181521,Gym / Fitness Center


In [35]:
#identify list coffee shops in Melbourne
venues = []
query = ['coffee shop']
for latitude, longtitude, name in zip(data_frame_of_coordinates['Latitude'], data_frame_of_coordinates['Longitude'], data_frame_of_coordinates['Suburb']):
    url = f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={latitude},{longtitude}&query={query}&radius={radius}&limit={LIMIT}"
    results = requests.get(url).json()["response"]['groups'][0]['items']
    for venue in results:
        venues.append((latitude, 
                       longtitude,
                       venue['venue']['name'],
                       venue['venue']['location']['lat'],
                       venue['venue']['location']['lng'],
                       venue['venue']['categories'][0]['name']))

In [36]:
bar_data_frame = pd.DataFrame(venues)
bar_data_frame.columns = ['Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
bar_data_frame

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,-37.814218,144.963161,Brother Baba Budan,-37.813445,144.962137,Coffee Shop
1,-37.814218,144.963161,Manchester Press,-37.813740,144.962132,Café
2,-37.814218,144.963161,Little Rogue,-37.810986,144.964059,Coffee Shop
3,-37.814218,144.963161,Hopetoun Tea Rooms,-37.815896,144.964634,Tea Room
4,-37.814218,144.963161,Dukes Coffee Roasters,-37.816763,144.966064,Coffee Shop
...,...,...,...,...,...,...
3551,51.426766,-0.052388,Sema's Cafe,51.438800,-0.033189,Café
3552,51.426766,-0.052388,Cafe Paradou,51.420522,-0.078655,Café
3553,41.877808,-87.902840,Stacy's Cafe,41.876360,-87.882689,Café
3554,41.877808,-87.902840,In House Coffe Shop!,41.886062,-87.899567,Coffee Shop


In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
#optain list of venues around Hawthorn
hawthorne_venues = getNearbyVenues(names=hawthorn_data['Suburb'],
                                   latitudes=hawthorn_data['Latitude'],
                                   longitudes=hawthorn_data['Longitude']
                                  )

Docklands
Abbotsford
Ivanhoe
Kew
Balwyn
Richmond
Hawthorn
Burwood
Toorak
Armadale
Malvern
Elwood


In [44]:
hawthorne_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbotsford,49.052116,-122.329479,The Reach Art Gallery,49.052734,-122.327687,Art Gallery
1,Abbotsford,49.052116,-122.329479,Greek Islands Restaurants,49.051039,-122.330412,Greek Restaurant
2,Abbotsford,49.052116,-122.329479,Original Joe's Restaurant & Bar,49.050402,-122.326861,Restaurant
3,Abbotsford,49.052116,-122.329479,Afterthoughts,49.049979,-122.323811,Dessert Shop
4,Abbotsford,49.052116,-122.329479,Booster Juice,49.050375,-122.327369,Smoothie Shop


In [48]:
hawthorne_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbotsford,22,22,22,22,22,22
Balwyn,14,14,14,14,14,14
Burwood,6,6,6,6,6,6
Docklands,48,48,48,48,48,48
Elwood,5,5,5,5,5,5
Hawthorn,7,7,7,7,7,7
Ivanhoe,7,7,7,7,7,7
Kew,27,27,27,27,27,27
Malvern,7,7,7,7,7,7


In [49]:
# one hot encoding
hawthorne_onehot = pd.get_dummies(hawthorne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hawthorne_onehot['Neighborhood'] = hawthorne_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hawthorne_onehot.columns[-1]] + list(hawthorne_onehot.columns[:-1])
hawthorne_onehot = hawthorne_onehot[fixed_columns]

hawthorne_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Boat or Ferry,...,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Tourist Information Center,Trail,Tram Station,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Docklands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Docklands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Docklands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Docklands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Docklands,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
hawthorne_grouped = hawthorne_onehot.groupby('Neighborhood').mean().reset_index()
hawthorne_grouped

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Boat or Ferry,...,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Tourist Information Center,Trail,Tram Station,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Abbotsford,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Balwyn,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Burwood,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Docklands,0.000000,0.000000,0.041667,0.000000,0.000000,0.020833,0.000000,0.000000,0.020833,...,0.041667,0.020833,0.020833,0.020833,0.000000,0.000000,0.020833,0.020833,0.000000,0.000000
4,Elwood,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Hawthorn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000
6,Ivanhoe,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.142857,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Kew,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000
8,Malvern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Richmond,0.046154,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.015385


In [53]:
#identify top 5 venues in each neighborhood
num_top_venues = 5

for hood in hawthorne_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = hawthorne_grouped[hawthorne_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbotsford----
              venue  freq
0  Greek Restaurant  0.09
1     Smoothie Shop  0.05
2   Supplement Shop  0.05
3               Gym  0.05
4    Discount Store  0.05


----Balwyn----
                  venue  freq
0                  Café  0.14
1  Gym / Fitness Center  0.07
2        Sandwich Place  0.07
3     Convenience Store  0.07
4           Supermarket  0.07


----Burwood----
                venue  freq
0  Italian Restaurant  0.17
1     Thai Restaurant  0.17
2  Chinese Restaurant  0.17
3       Grocery Store  0.17
4        Burger Joint  0.17


----Docklands----
                       venue  freq
0  Middle Eastern Restaurant  0.06
1         Italian Restaurant  0.06
2                      Hotel  0.04
3                       Café  0.04
4           Asian Restaurant  0.04


----Elwood----
               venue  freq
0      Grocery Store   0.2
1        Pizza Place   0.2
2  Convenience Store   0.2
3               Park   0.2
4        Gas Station   0.2


----Hawthorn----
              

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hawthorne_grouped['Neighborhood']

for ind in np.arange(hawthorne_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hawthorne_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbotsford,Greek Restaurant,Smoothie Shop,Supplement Shop,Gym,Discount Store,Hardware Store,Coffee Shop,Pizza Place,Paper / Office Supplies Store,Farmers Market
1,Balwyn,Café,Gym / Fitness Center,Sandwich Place,Convenience Store,Supermarket,Pet Store,Farmers Market,Pharmacy,Japanese Restaurant,Bakery
2,Burwood,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Grocery Store,Burger Joint,Pub,Malay Restaurant,Noodle House,Playground,Pizza Place
3,Docklands,Middle Eastern Restaurant,Italian Restaurant,Hotel,Café,Asian Restaurant,Steakhouse,Flea Market,Chinese Restaurant,Restaurant,Sandwich Place
4,Elwood,Grocery Store,Pizza Place,Convenience Store,Park,Gas Station,New American Restaurant,Pier,Pharmacy,Pet Store,Performing Arts Venue
5,Hawthorn,Café,Tram Station,Convenience Store,Grocery Store,Fish & Chips Shop,Lounge,Malay Restaurant,Pool Hall,Playground,Pizza Place
6,Ivanhoe,Park,Food,Automotive Shop,Playground,Bar,Home Service,American Restaurant,Noodle House,Pizza Place,Pier
7,Kew,Garden,Playground,Botanical Garden,Café,Monument / Landmark,Scenic Lookout,Ice Cream Shop,Art Gallery,Trail,Sculpture Garden
8,Malvern,Pub,Supermarket,Hotel Bar,Furniture / Home Store,Coffee Shop,New American Restaurant,Pizza Place,Pier,Pharmacy,Pet Store
9,Richmond,Coffee Shop,Sandwich Place,Hotel,American Restaurant,History Museum,Food Truck,Lounge,Italian Restaurant,Deli / Bodega,Scenic Lookout


In [ ]:
#Since Margot's life evoles around fitness and cafes, Balwyn is a wonderful choice for her.